In [4]:
import pandas as pd
import ast
import requests
import time

In [2]:
# CSV 파일 로드
df = pd.read_csv("hehe.csv")
df.head(3)  # 앞부분만 미리보기

,user_input,total_requirements,project_info,ERD_data
0,"[{'projectName': '스터디 그룹 성과 분석 도구', 'projectTa...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '스터디 그룹 성과 분석 도구', ...","{'erd_tables': [{'name': 'users', 'erd_columns..."
1,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '기억의 다리', 'category...","{'erd_tables': [{'name': 'Users', 'erd_columns..."
2,"[{'projectName': '장애인 친화 대중교통 안내 앱', 'projectT...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '장애인 친화 대중교통 안내 앱',...","{'erd_tables': [{'name': 'Users', 'erd_columns..."


In [13]:
def parse_field(field):
    """문자열로 저장된 dict/list를 실제 파이썬 객체로 변환"""
    try:
        return str(ast.literal_eval(field))
    except Exception as e:
        print("변환 오류:", e)
        return None


In [17]:
base_url = "http://3.34.185.3:8000/"

def ERD_generate(user_info, requirement, project_info):
    data = {
        "project_overview": user_info,
        "requirements": requirement,
        "project_summury": project_info,
        "max_tokens": 4000,
        "temperature": 0.3,
        "model": "gpt-4o"
    }
    try:
        start_time = time.time()
        response = requests.post(
            url = base_url + "api/PJA/json_ERD/generate",
            json = data,
            headers= {"Content-Type": "application/json"}
        )
        elapsed = time.time() - start_time

        return response.json().get('json', None), elapsed
    
    except Exception as e:
        print("ERD_generate error:", e)
        return None, None

def new_ERD_generate(user_info, requirement, project_info):
    data = {
        "project_overview": user_info,
        "requirements": requirement,
        "project_summury": project_info,
        "max_tokens": 4000,
        "temperature": 0.3,
        "model": "ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyl6o"
    }
    try:
        start_time = time.time()
        response = requests.post(
            url = base_url + "api/PJA/json_ERD/generate",
            json = data,
            headers= {"Content-Type": "application/json"}
        )
        elapsed = time.time() - start_time

        return response.json().get('json', None), elapsed
    
    except Exception as e:
        print("new_ERD_generate error:", e)
        return None, None


In [18]:
# 0번(첫 번째) 프로젝트 샘플로 테스트
row = df.iloc[0]
user_input = parse_field(row['user_input'])
requirements = parse_field(row['total_requirements'])
project_info = parse_field(row['project_info'])

# 기본 모델 ERD
erd1, time1 = ERD_generate(user_input, requirements, project_info)
print("GPT-4o ERD 결과:", erd1)
print("GPT-4o 응답시간:", time1, "초")

# 파인튜닝 모델 ERD
erd2, time2 = new_ERD_generate(user_input, requirements, project_info)
print("Fine-tuned GPT-4o-mini ERD 결과:", erd2)
print("Fine-tuned 응답시간:", time2, "초")


GPT-4o ERD 결과: {'erd_tables': [{'name': 'users', 'erd_columns': [{'name': 'user_id', 'data_type': 'serial', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'email', 'data_type': 'varchar(255)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'password', 'data_type': 'varchar(255)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'name', 'data_type': 'varchar(255)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'email_verified', 'data_type': 'boolean', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}]}, {'name': 'study_groups', 'erd_columns': [{'name': 'group_id', 'data_type': 'serial', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'group_name', 'data_type': 'varchar(255)', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'created_by', 'data_type': 'integer', 'is

In [6]:
results = []
for idx, row in df.iterrows():
    user_input = parse_field(row['user_input'])
    requirements = parse_field(row['total_requirements'])
    project_info = parse_field(row['project_info'])

    erd1, time1 = ERD_generate(user_input, requirements, project_info)
    erd2, time2 = new_ERD_generate(user_input, requirements, project_info)

    results.append({
        "project_title": project_info['project_info']['title'],
        "gpt-4o_erd": erd1,
        "gpt-4o_time": time1,
        "finetuned_erd": erd2,
        "finetuned_time": time2
    })
    print(f"[{idx+1}] {project_info['project_info']['title']} 완료")

result_df = pd.DataFrame(results)
result_df.head()


[1] 스터디 그룹 성과 분석 도구 완료
[2] 기억의 다리 완료
[3] 장애인 친화 대중교통 안내 앱 완료
[4] 가상 피팅룸 서비스 완료
[5] 스마트 건강 관리 비서 완료
[6] 그린 챌린지 완료
[7] 스마트 학습 기록 관리자 완료
[8] 스마트 영양소 분석기 완료
[9] 감정 기록 매니저 완료
[10] 재무관리 도우미(Financial Buddy) 완료
[11] 결혼 준비 도우미 완료
[12] 중고 물품 대여 매칭 플랫폼 완료
[13] EcoSense: 실시간 환경 모니터링 시스템 완료
[14] 스마트 공기질 분석 시스템 완료
[15] 비접촉식 감정 인식 시스템 완료
[16] 스마트 공기질 관리 시스템 완료
[17] 필체 스타일 변환기 완료
[18] 스마트 에너지 관리 시스템 완료
[19] 손 제스처 기반 금융 보안 인증 시스템 완료
[20] Smart Consumer Insight Platform 완료
[21] 실시간 소비자 의견 분석 플랫폼 완료
[22] 스마트 교통 관리 시스템 완료
[23] 스마트 실내 공기 질 관리 시스템 완료
[24] 약 복용 리마인더 및 모니터링 시스템 완료
[25] 스마트 피트니스 코치 완료
[26] 스마트 음성 안내 내비게이션 완료
[27] 출장 스트레스 관리 플랫폼 완료
[28] 축제 방문자 흐름 최적화 시스템 완료
[29] 전통시장 스마트 재고 관리 시스템 완료
[30] 스마트 출결 관리 시스템 완료
[31] 스마트 에너지 모니터링 시스템 완료
[32] UrbanGreen Monitor 완료
[33] BookClub 완료
[34] ExpenseAnalyzer 완료
[35] MealPrepMaster 완료
[36] EcoTrack 완료
[37] 그린 케어: 스마트 식물 관리 솔루션 완료
[38] SmartTask 도우미 완료
[39] 리뷰북 커뮤니티 완료
[40] StudyTracker 완료
[41] 스마트학습관리 완료
[42] 스마트 농업 분석기 완료
[43] TeamSync 완료
[44] 리뷰 진위 확인 시스템 완

,project_title,gpt-4o_erd,gpt-4o_time,finetuned_erd,finetuned_time
0,스터디 그룹 성과 분석 도구,None,0.022335,None,0.020827
1,기억의 다리,None,0.020424,None,0.022652
2,장애인 친화 대중교통 안내 앱,None,0.019332,None,0.026310
3,가상 피팅룸 서비스,None,0.015917,None,0.020398
4,스마트 건강 관리 비서,None,0.018303,None,0.019620


In [7]:
result_df.to_csv("erd_comparison_results.csv", index=False)
print("비교 결과가 erd_comparison_results.csv에 저장되었습니다.")

비교 결과가 erd_comparison_results.csv에 저장되었습니다.


In [8]:
import json

# 첫 번째 프로젝트 예시로 구조 비교
print("GPT-4o ERD Table 수:", len(erd1['erd_tables']) if erd1 else "N/A")
print("Fine-tuned ERD Table 수:", len(erd2['erd_tables']) if erd2 else "N/A")

# (더 복잡한 비교/시각화는 이후 추가 가능)


GPT-4o ERD Table 수: N/A
Fine-tuned ERD Table 수: N/A


In [9]:
# CSV 파일 로드
df = pd.read_csv("erd_comparison_results.csv")
df.head(3)  # 앞부분만 미리보기

,project_title,gpt-4o_erd,gpt-4o_time,finetuned_erd,finetuned_time
0,스터디 그룹 성과 분석 도구,NaN,0.022335,NaN,0.020827
1,기억의 다리,NaN,0.020424,NaN,0.022652
2,장애인 친화 대중교통 안내 앱,NaN,0.019332,NaN,0.026310


In [ ]:
"""
파인튜닝 모델 vs GPT-4o 성능 비교 테스트
한국어 요구사항 분석 태스크 평가

파일명: benchmark_pja_vs_gpt4o.py
목적: 파인튜닝된 모델과 GPT-4o의 성능을 정량적으로 비교 분석
"""

import os
import json
import time
import pandas as pd
from tqdm import tqdm
import openai
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np

# ========================================================================================
# 전역 설정 및 상수 정의
# ========================================================================================

# 테스트할 모델 정의
FINETUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyl6o"  # 파인튜닝된 모델
BASELINE_MODEL = "gpt-4o"  # 비교 대상인 기본 GPT-4o 모델

# API 호출 설정
TEMPERATURE = 0.2  # 낮은 온도로 일관성 있는 응답 생성
CSV_FILE_PATH = "hehe.csv"  # 테스트 데이터가 들어있는 CSV 파일 경로

# 한국어 시스템 프롬프트 - 요구사항 분석 전문가 역할 정의
SYSTEM_PROMPT = """당신은 숙련된 소프트웨어 요구사항 분석 전문가입니다. 
사용자의 프로젝트 설명을 바탕으로 체계적이고 구체적인 요구사항 목록을 작성해주세요.
기능적 요구사항과 성능 요구사항을 구분하여 작성하고, 각 요구사항은 명확하고 측정 가능해야 합니다."""

# ========================================================================================
# 핵심 함수 정의
# ========================================================================================

def openai_chat_completion(model, user_input, project_info):
    """
    OpenAI Chat Completion API를 호출하는 함수
    
    Args:
        model (str): 사용할 모델명 (파인튜닝 모델 또는 GPT-4o)
        user_input (str): 사용자가 입력한 요구사항 설명
        project_info (str): 프로젝트 정보 (JSON 형태의 문자열)
    
    Returns:
        str: 모델이 생성한 요구사항 목록 텍스트
    """
    try:
        # 시스템 메시지와 사용자 메시지 구성
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
프로젝트 정보:
{project_info}

사용자 요구사항:
{user_input}

위 정보를 바탕으로 체계적인 요구사항 목록을 작성해주세요. 
기능적 요구사항과 성능 요구사항으로 구분하여 불릿 포인트 형태로 작성해주세요.
"""}
        ]
        
        # OpenAI API 호출
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMPERATURE,  # 일관성 있는 응답을 위한 낮은 온도
            max_tokens=2000  # 충분한 길이의 응답을 위한 토큰 제한
        )
        
        # 응답 텍스트 추출 및 공백 제거
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        # API 호출 실패시 에러 로그 출력 및 빈 문자열 반환
        print(f"API 호출 오류 ({model}): {e}")
        return ""

def calculate_metrics(predictions, references):
    """
    모델 성능을 평가하는 다양한 메트릭을 계산하는 함수
    
    Args:
        predictions (list): 모델이 생성한 텍스트 목록
        references (list): 정답(ground truth) 텍스트 목록
    
    Returns:
        dict: 계산된 성능 메트릭들 (BLEU, ROUGE-L, 정확일치율)
    """
    
    # 1. BLEU 점수 계산 - n-gram 기반 유사도 측정
    # corpus_bleu는 전체 코퍼스에 대한 BLEU 점수를 계산
    bleu_score = corpus_bleu(predictions, [references]).score
    
    # 2. ROUGE-L 점수 계산 - 최장 공통 부분 수열 기반 유사도 측정
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, pred)['rougeL'].fmeasure 
                   for ref, pred in zip(references, predictions)]
    rouge_l_score = np.mean(rouge_scores) * 100  # 백분율로 변환
    
    # 3. 정확 일치율 계산 - 완전히 동일한 출력의 비율
    exact_matches = sum(1 for p, r in zip(predictions, references) 
                       if p.strip() == r.strip())
    exact_match_rate = (exact_matches / len(references)) * 100
    
    # 결과를 딕셔너리로 반환 (소수점 둘째 자리까지)
    return {
        "BLEU": round(bleu_score, 2),
        "ROUGE-L": round(rouge_l_score, 2),
        "정확일치율(%)": round(exact_match_rate, 2)
    }

def run_evaluation():
    """
    메인 평가 실행 함수 - 전체 성능 비교 프로세스를 관리
    """
    print("📊 파인튜닝 모델 vs GPT-4o 성능 비교 시작")
    print("=" * 60)
    
    # ========================================================================================
    # 1. 데이터 로드 및 검증
    # ========================================================================================
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"✅ 데이터 로드 완료: {len(df)}개 샘플")
        
        # 필수 컬럼 존재 여부 확인
        required_columns = ['user_input', 'project_info', 'total_requirements']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"필수 컬럼이 누락되었습니다: {missing_columns}")
            
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return
    
    # ========================================================================================
    # 2. 모델별 추론 실행
    # ========================================================================================
    
    # 각 모델의 결과를 저장할 딕셔너리 초기화
    results = {
        FINETUNED_MODEL: [],  # 파인튜닝 모델 결과
        BASELINE_MODEL: []    # GPT-4o 결과
    }
    
    print(f"\n🔄 {len(df)}개 샘플에 대해 모델 추론 시작...")
    
    # 각 데이터 샘플에 대해 두 모델로 추론 수행
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="모델 추론 진행"):
        user_input = row['user_input']        # 사용자 요구사항 설명
        project_info = row['project_info']    # 프로젝트 정보
        
        # 파인튜닝 모델로 추론
        print(f"  📝 샘플 {idx+1}: 파인튜닝 모델 추론 중...")
        ft_result = openai_chat_completion(FINETUNED_MODEL, user_input, project_info)
        results[FINETUNED_MODEL].append(ft_result)
        
        # GPT-4o 모델로 추론  
        print(f"  📝 샘플 {idx+1}: GPT-4o 추론 중...")
        gpt4o_result = openai_chat_completion(BASELINE_MODEL, user_input, project_info)
        results[BASELINE_MODEL].append(gpt4o_result)
        
        # API 호출 간격 조정 (레이트 리미트 방지)
        # 0.5초 대기로 안정적인 API 호출 보장
        time.sleep(0.5)
        
        # 진행 상황 출력 (10개마다)
        if (idx + 1) % 10 == 0:
            print(f"  ✅ {idx + 1}/{len(df)} 샘플 완료")
    
    # ========================================================================================
    # 3. 성능 메트릭 계산 및 결과 출력
    # ========================================================================================
    
    print("\n📈 성능 평가 결과")
    print("=" * 60)
    
    # 정답 데이터 (ground truth) 추출
    references = df['ERD_data'].tolist()
    
    # 각 모델별로 성능 메트릭 계산
    final_results = {}
    for model_name, predictions in results.items():
        # 모델명을 한국어로 표시하기 위한 변환
        model_display_name = "파인튜닝 모델" if "ft:" in model_name else "GPT-4o"
        
        # 성능 메트릭 계산
        metrics = calculate_metrics(predictions, references)
        
        # 결과 출력
        print(f"\n🤖 {model_display_name}")
        print(f"   BLEU 점수: {metrics['BLEU']}")
        print(f"   ROUGE-L 점수: {metrics['ROUGE-L']}")
        print(f"   정확 일치율: {metrics['정확일치율(%)']}%")
        
        # 최종 결과 딕셔너리에 저장
        model_key = "파인튜닝_모델" if "ft:" in model_name else "GPT-4o"
        final_results[model_key] = metrics
    
    # ========================================================================================
    # 4. 결과 저장
    # ========================================================================================
    
    # JSON 형태로 성능 비교 결과 저장
    try:
        with open("성능비교_결과.json", "w", encoding="utf-8") as f:
            json.dump(final_results, f, ensure_ascii=False, indent=2)
        print(f"\n💾 결과가 '성능비교_결과.json' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"❌ 결과 저장 실패: {e}")
    
    # 상세한 출력 결과를 CSV로 저장
    try:
        output_df = df.copy()
        output_df['파인튜닝_모델_출력'] = results[FINETUNED_MODEL]
        output_df['GPT4o_출력'] = results[BASELINE_MODEL]
        output_df.to_csv("모델_출력_비교.csv", index=False, encoding="utf-8")
        print(f"📄 상세 출력 결과가 '모델_출력_비교.csv' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"❌ 상세 결과 저장 실패: {e}")
    
    # ========================================================================================
    # 5. 성능 차이 분석
    # ========================================================================================
    
    print(f"\n📊 성능 차이 분석")
    print("=" * 60)
    
    if len(final_results) == 2:
        ft_bleu = final_results['파인튜닝_모델']['BLEU']
        gpt4o_bleu = final_results['GPT-4o']['BLEU']
        bleu_diff = ft_bleu - gpt4o_bleu
        
        ft_rouge = final_results['파인튜닝_모델']['ROUGE-L']
        gpt4o_rouge = final_results['GPT-4o']['ROUGE-L']
        rouge_diff = ft_rouge - gpt4o_rouge
        
        print(f"BLEU 점수 차이: {bleu_diff:+.2f} ({'향상' if bleu_diff > 0 else '저하'})")
        print(f"ROUGE-L 점수 차이: {rouge_diff:+.2f} ({'향상' if rouge_diff > 0 else '저하'})")
        
        # 성능 개선 여부 판단
        if bleu_diff > 0 and rouge_diff > 0:
            print("🎉 파인튜닝 모델이 모든 지표에서 우수한 성능을 보입니다!")
        elif bleu_diff < 0 and rouge_diff < 0:
            print("⚠️  파인튜닝 모델의 성능이 기본 모델보다 낮습니다.")
        else:
            print("📝 일부 지표에서만 성능 향상이 있습니다.")

# ========================================================================================
# 메인 실행 부분
# ========================================================================================

if __name__ == "__main__":
    # OpenAI API 키 확인
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY 환경변수를 설정해주세요.")
        print("   export OPENAI_API_KEY='your-api-key-here'")
        exit(1)
    
    print("🚀 성능 비교 테스트를 시작합니다...")
    print(f"📁 테스트 데이터: {CSV_FILE_PATH}")
    print(f"🤖 파인튜닝 모델: {FINETUNED_MODEL}")
    print(f"🤖 기본 모델: {BASELINE_MODEL}")
    print(f"🌡️  온도 설정: {TEMPERATURE}")
    
    # 메인 평가 함수 실행
    run_evaluation()
    
    print("\n✅ 성능 비교 테스트가 완료되었습니다!")


In [23]:
metrics

NameError: name 'metrics' is not defined

In [25]:
"""
파인튜닝 모델 vs GPT-4o 성능 비교 테스트 (개선 버전)
한국어 요구사항 분석 태스크 평가

주요 개선사항:
1. 정답 데이터 참조 오류 수정 (ERD_data -> total_requirements)
2. JSON 형태 데이터 처리 로직 추가
3. 더 적합한 평가 메트릭 추가 (의미적 유사도)
4. 에러 처리 강화
"""

import os
import json
import time
import pandas as pd
from tqdm import tqdm
import openai
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ========================================================================================
# 전역 설정 및 상수 정의
# ========================================================================================

# 테스트할 모델 정의
FINETUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyrDW:ckpt-step-124"
BASELINE_MODEL = "gpt-4o"

# API 호출 설정
TEMPERATURE = 0.2
CSV_FILE_PATH = "hehe.csv"

# 한국어 ERD 생성 전문가 시스템 프롬프트 (파인튜닝 목적에 맞게 수정)
SYSTEM_PROMPT = """당신은 숙련된 데이터베이스 설계 전문가입니다. 
사용자의 프로젝트 설명과 요구사항을 바탕으로 ERD(Entity Relationship Diagram) 데이터를 생성해주세요.
각 테이블의 구조, 컬럼 정보, 관계를 명확하게 정의해야 합니다.
JSON 형태로 구조화된 ERD 정보를 제공해주세요."""

# 의미적 유사도 계산을 위한 모델 (한국어 지원)
try:
    semantic_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    SEMANTIC_SIMILARITY_AVAILABLE = True
except:
    print("⚠️ 의미적 유사도 모델을 로드할 수 없습니다. 해당 메트릭은 제외됩니다.")
    SEMANTIC_SIMILARITY_AVAILABLE = False

# ========================================================================================
# 유틸리티 함수
# ========================================================================================

def safe_parse_json_string(json_str):
    """
    JSON 문자열을 안전하게 파싱하는 함수
    """
    if pd.isna(json_str) or json_str == '':
        return None
    
    try:
        # 문자열이 이미 딕셔너리/리스트인 경우
        if isinstance(json_str, (dict, list)):
            return json_str
        
        # 문자열을 JSON으로 파싱 시도
        if isinstance(json_str, str):
            # ast.literal_eval 먼저 시도 (Python 리터럴)
            try:
                return ast.literal_eval(json_str)
            except:
                # json.loads 시도
                return json.loads(json_str)
    except Exception as e:
        print(f"JSON 파싱 오류: {e}")
        return json_str  # 원본 문자열 반환

def format_data_for_prompt(data):
    """
    데이터를 프롬프트에 적합한 형태로 포맷팅
    """
    if isinstance(data, str):
        return data
    elif isinstance(data, (dict, list)):
        return json.dumps(data, ensure_ascii=False, indent=2)
    else:
        return str(data)

def extract_text_content(data):
    """
    JSON 데이터에서 텍스트 내용을 추출하여 비교 가능한 형태로 변환
    """
    if isinstance(data, str):
        return data.strip()
    elif isinstance(data, dict):
        # 딕셔너리에서 주요 텍스트 내용 추출
        text_parts = []
        for key, value in data.items():
            if isinstance(value, str):
                text_parts.append(f"{key}: {value}")
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, str):
                        text_parts.append(item)
                    elif isinstance(item, dict):
                        text_parts.append(json.dumps(item, ensure_ascii=False))
        return " ".join(text_parts)
    elif isinstance(data, list):
        # 리스트에서 텍스트 내용 추출
        text_parts = []
        for item in data:
            if isinstance(item, str):
                text_parts.append(item)
            elif isinstance(item, dict):
                text_parts.append(json.dumps(item, ensure_ascii=False))
        return " ".join(text_parts)
    else:
        return str(data)

# ========================================================================================
# 핵심 함수 정의
# ========================================================================================

def openai_chat_completion(model, user_input, project_info):
    """
    OpenAI Chat Completion API를 호출하는 함수
    """
    try:
        # 데이터를 프롬프트 형태로 포맷팅
        formatted_project_info = format_data_for_prompt(project_info)
        formatted_user_input = format_data_for_prompt(user_input)
        
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
프로젝트 정보:
{formatted_project_info}

요구사항:
{formatted_user_input}

위 정보를 바탕으로 ERD 데이터를 생성해주세요.
테이블 구조, 컬럼 정보, 관계를 포함한 JSON 형태로 제공해주세요.
"""}
        ]
        
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMPERATURE,
            max_tokens=2000
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"API 호출 오류 ({model}): {e}")
        return ""

def calculate_semantic_similarity(predictions, references):
    """
    의미적 유사도를 계산하는 함수
    """
    if not SEMANTIC_SIMILARITY_AVAILABLE:
        return 0.0
    
    try:
        # 텍스트 내용 추출
        pred_texts = [extract_text_content(pred) for pred in predictions]
        ref_texts = [extract_text_content(ref) for ref in references]
        
        # 임베딩 생성
        pred_embeddings = semantic_model.encode(pred_texts)
        ref_embeddings = semantic_model.encode(ref_texts)
        
        # 코사인 유사도 계산
        similarities = []
        for pred_emb, ref_emb in zip(pred_embeddings, ref_embeddings):
            similarity = cosine_similarity([pred_emb], [ref_emb])[0][0]
            similarities.append(similarity)
        
        return np.mean(similarities) * 100
    except Exception as e:
        print(f"의미적 유사도 계산 오류: {e}")
        return 0.0

def calculate_metrics(predictions, references):
    """
    모델 성능을 평가하는 다양한 메트릭을 계산하는 함수
    """
    # 텍스트 내용 추출
    pred_texts = [extract_text_content(pred) for pred in predictions]
    ref_texts = [extract_text_content(ref) for ref in references]
    
    # 빈 텍스트 필터링
    valid_pairs = [(p, r) for p, r in zip(pred_texts, ref_texts) if p.strip() and r.strip()]
    
    if not valid_pairs:
        return {
            "BLEU": 0.0,
            "ROUGE-L": 0.0,
            "정확일치율(%)": 0.0,
            "의미적_유사도(%)": 0.0
        }
    
    valid_preds, valid_refs = zip(*valid_pairs)
    
    # 1. BLEU 점수 계산
    try:
        bleu_score = corpus_bleu(list(valid_preds), [list(valid_refs)]).score
    except:
        bleu_score = 0.0
    
    # 2. ROUGE-L 점수 계산
    try:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        rouge_scores = [scorer.score(ref, pred)['rougeL'].fmeasure 
                       for ref, pred in zip(valid_refs, valid_preds)]
        rouge_l_score = np.mean(rouge_scores) * 100
    except:
        rouge_l_score = 0.0
    
    # 3. 정확 일치율 계산
    exact_matches = sum(1 for p, r in zip(valid_preds, valid_refs) 
                       if p.strip() == r.strip())
    exact_match_rate = (exact_matches / len(valid_pairs)) * 100
    
    # 4. 의미적 유사도 계산
    semantic_similarity = calculate_semantic_similarity(predictions, references)
    
    return {
        "BLEU": round(bleu_score, 2),
        "ROUGE-L": round(rouge_l_score, 2),
        "정확일치율(%)": round(exact_match_rate, 2),
        "의미적_유사도(%)": round(semantic_similarity, 2)
    }

def run_evaluation():
    """
    메인 평가 실행 함수
    """
    print("📊 파인튜닝 모델 vs GPT-4o 성능 비교 시작 (개선 버전)")
    print("=" * 60)
    
    # ========================================================================================
    # 1. 데이터 로드 및 검증
    # ========================================================================================
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"✅ 데이터 로드 완료: {len(df)}개 샘플")
        
        # 컬럼 정보 출력
        print(f"📋 CSV 컬럼: {list(df.columns)}")
        
        # 필수 컬럼 확인 (수정된 버전)
        required_columns = ['user_input', 'project_info', 'total_requirements', 'ERD_data']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"⚠️ 일부 컬럼이 누락되었습니다: {missing_columns}")
            print("사용 가능한 컬럼으로 진행합니다.")
            
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return
    
    # 데이터 전처리
    print("\n🔄 데이터 전처리 중...")
    
    # JSON 문자열을 파싱
    for col in ['user_input', 'project_info', 'total_requirements', 'ERD_data']:
        if col in df.columns:
            df[col] = df[col].apply(safe_parse_json_string)
    
    # 처음 몇 개 샘플의 데이터 형태 확인
    print("\n📋 데이터 샘플 확인:")
    for i, row in df.head(2).iterrows():
        print(f"\n샘플 {i+1}:")
        for col in df.columns:
            print(f"  {col}: {type(row[col])} - {str(row[col])[:100]}...")
    
    # ========================================================================================
    # 2. 모델별 추론 실행 (소규모 테스트)
    # ========================================================================================
    
    # 테스트할 샘플 수 제한 (비용 절약)
    test_samples = min(5, len(df))  # 최대 5개 샘플로 테스트
    test_df = df.head(test_samples).copy()
    
    print(f"\n🔄 {test_samples}개 샘플에 대해 모델 추론 시작...")
    
    results = {
        FINETUNED_MODEL: [],
        BASELINE_MODEL: []
    }
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="모델 추론 진행"):
        user_input = row['user_input']
        project_info = row['project_info']
        
        print(f"\n📝 샘플 {idx+1} 처리 중...")
        print(f"   입력 타입: {type(user_input)}")
        print(f"   프로젝트 정보 타입: {type(project_info)}")
        
        # 파인튜닝 모델 추론
        ft_result = openai_chat_completion(FINETUNED_MODEL, user_input, project_info)
        results[FINETUNED_MODEL].append(ft_result)
        
        # GPT-4o 추론
        gpt4o_result = openai_chat_completion(BASELINE_MODEL, user_input, project_info)
        results[BASELINE_MODEL].append(gpt4o_result)
        
        time.sleep(1.0)  # API 레이트 리미트 방지
    
    # ========================================================================================
    # 3. 성능 메트릭 계산
    # ========================================================================================
    
    print("\n📈 성능 평가 결과")
    print("=" * 60)
    
    # 정답 데이터 - ERD_data를 정답으로 사용 (파인튜닝 목적에 맞게)
    references = test_df['ERD_data'].tolist()
    
    print(f"📊 정답 데이터 형태 확인:")
    for i, ref in enumerate(references[:2]):
        print(f"  정답 {i+1}: {type(ref)} - {str(ref)[:200]}...")
    
    final_results = {}
    for model_name, predictions in results.items():
        model_display_name = "파인튜닝 모델" if "ft:" in model_name else "GPT-4o"
        
        print(f"\n🔍 {model_display_name} 출력 확인:")
        for i, pred in enumerate(predictions[:2]):
            print(f"  출력 {i+1}: {str(pred)[:200]}...")
        
        metrics = calculate_metrics(predictions, references)
        
        print(f"\n🤖 {model_display_name} 성능:")
        for metric_name, value in metrics.items():
            print(f"   {metric_name}: {value}")
        
        model_key = "파인튜닝_모델" if "ft:" in model_name else "GPT-4o"
        final_results[model_key] = metrics
    
    # ========================================================================================
    # 4. 결과 저장 및 분석
    # ========================================================================================
    
    # 결과 저장
    try:
        with open("성능비교_결과_개선.json", "w", encoding="utf-8") as f:
            json.dump(final_results, f, ensure_ascii=False, indent=2)
        print(f"\n💾 결과가 '성능비교_결과_개선.json' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"❌ 결과 저장 실패: {e}")
    
    # 상세 결과 저장
    try:
        output_df = test_df.copy()
        output_df['파인튜닝_모델_출력'] = results[FINETUNED_MODEL]
        output_df['GPT4o_출력'] = results[BASELINE_MODEL]
        output_df.to_csv("모델_출력_비교_개선.csv", index=False, encoding="utf-8")
        print(f"📄 상세 출력이 '모델_출력_비교_개선.csv'로 저장되었습니다.")
    except Exception as e:
        print(f"❌ 상세 결과 저장 실패: {e}")
    
    # 성능 차이 분석
    if len(final_results) == 2:
        print(f"\n📊 성능 차이 분석")
        print("=" * 60)
        
        ft_metrics = final_results['파인튜닝_모델']
        gpt4o_metrics = final_results['GPT-4o']
        
        for metric in ['BLEU', 'ROUGE-L', '의미적_유사도(%)']:
            if metric in ft_metrics and metric in gpt4o_metrics:
                diff = ft_metrics[metric] - gpt4o_metrics[metric]
                print(f"{metric} 차이: {diff:+.2f} ({'향상' if diff > 0 else '저하'})")
        
        # 종합 평가
        improvements = sum(1 for metric in ['BLEU', 'ROUGE-L', '의미적_유사도(%)'] 
                          if ft_metrics.get(metric, 0) > gpt4o_metrics.get(metric, 0))
        
        if improvements >= 2:
            print("🎉 파인튜닝 모델이 전반적으로 우수한 성능을 보입니다!")
        elif improvements == 1:
            print("📝 일부 지표에서 성능 향상이 있습니다.")
        else:
            print("⚠️ 파인튜닝 효과가 제한적입니다. 데이터나 방법론 재검토가 필요할 수 있습니다.")

# ========================================================================================
# 메인 실행 부분
# ========================================================================================

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY 환경변수를 설정해주세요.")
        print("   export OPENAI_API_KEY='your-api-key-here'")
        exit(1)
    
    print("🚀 개선된 성능 비교 테스트를 시작합니다...")
    print(f"📁 테스트 데이터: {CSV_FILE_PATH}")
    print(f"🤖 파인튜닝 모델: {FINETUNED_MODEL}")
    print(f"🤖 기본 모델: {BASELINE_MODEL}")
    print(f"🌡️ 온도 설정: {TEMPERATURE}")
    
    run_evaluation()
    
    print("\n✅ 개선된 성능 비교 테스트가 완료되었습니다!")

c:\Users\mir96\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


🚀 개선된 성능 비교 테스트를 시작합니다...
📁 테스트 데이터: hehe.csv
🤖 파인튜닝 모델: ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyrDW:ckpt-step-124
🤖 기본 모델: gpt-4o
🌡️ 온도 설정: 0.2
📊 파인튜닝 모델 vs GPT-4o 성능 비교 시작 (개선 버전)
✅ 데이터 로드 완료: 138개 샘플
📋 CSV 컬럼: ['user_input', 'total_requirements', 'project_info', 'ERD_data']

🔄 데이터 전처리 중...

📋 데이터 샘플 확인:

샘플 1:
  user_input: <class 'list'> - [{'projectName': '스터디 그룹 성과 분석 도구', 'projectTarget': '학습자 및 교육자', 'mainFunction': ['스터디 그룹 성과 시각화', ...
  total_requirements: <class 'list'> - [{'requirementType': 'FUNCTIONAL', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정 생성의 안전성을 높일 수 있어야 한다.'}, {'...
  project_info: <class 'dict'> - {'project_info': {'title': '스터디 그룹 성과 분석 도구', 'category': '웹앱', 'target_users': ['학습자', '교육자', '스터디 ...
  ERD_data: <class 'dict'> - {'erd_tables': [{'name': 'users', 'erd_columns': [{'name': 'user_id', 'data_type': 'SERIAL', 'is_pri...

샘플 2:
  user_input: <class 'list'> - [{'projectName': '기억의 다리', 'projectTarget': '치매 환자와 그 가족', 'mainFunction': ['추

모델 추론 진행:   0%|          | 0/5 [00:00<?, ?it/s]


📝 샘플 1 처리 중...
   입력 타입: <class 'list'>
   프로젝트 정보 타입: <class 'dict'>


모델 추론 진행:  20%|██        | 1/5 [00:37<02:30, 37.66s/it]

...


📝 샘플 4 처리 중...
   입력 타입: <class 'list'>
   프로젝트 정보 타입: <class 'dict'>


모델 추론 진행:  80%|████████  | 4/5 [02:30<00:38, 38.28s/it]


📝 샘플 5 처리 중...
   입력 타입: <class 'list'>
   프로젝트 정보 타입: <class 'dict'>


모델 추론 진행: 100%|██████████| 5/5 [03:10<00:00, 38.00s/it]



📈 성능 평가 결과
📊 정답 데이터 형태 확인:
  정답 1: <class 'dict'> - {'erd_tables': [{'name': 'users', 'erd_columns': [{'name': 'user_id', 'data_type': 'SERIAL', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'email', 'data_type': 'VAR...
  정답 2: <class 'dict'> - {'erd_tables': [{'name': 'Users', 'erd_columns': [{'name': 'user_id', 'data_type': 'ObjectId', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'username', 'data_type':...

🔍 파인튜닝 모델 출력 확인:
  출력 1: 아래는 "스터디 그룹 성과 분석 도구" 프로젝트에 대한 ERD(Entity Relationship Diagram) 정보를 JSON 형태로 제공한 것입니다. 이 ERD는 사용자, 스터디 그룹, 퀴즈, 과제, 공지사항 등과 관련된 테이블 구조와 관계를 포함하고 있습니다.

```json
{
  "erd_tables": [
    {
      "name": "...
  출력 2: 아래는 "기억의 다리" 프로젝트를 위한 ERD(Entity Relationship Diagram) 데이터입니다. 이 ERD는 사용자의 요구사항을 기반으로 테이블 구조, 컬럼 정보, 그리고 관계를 포함하고 있습니다.

```json
{
  "erd_tables": [
    {
      "name": "Users",
      "columns": [
   ...

🤖 파인튜닝 모델 성능:
   BLEU: 69.09
   ROUGE-L: 81.11
   정확일치율(%): 0.0
   의미적_

In [7]:
"""
3개 모델 성능 비교: 파인튜닝 vs GPT-4o-mini vs GPT-4o
파인튜닝 효과를 명확하게 측정하기 위한 완전한 비교
"""

import os
import json
import time
import pandas as pd
from tqdm import tqdm
import openai
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ========================================================================================
# 전역 설정 및 상수 정의
# ========================================================================================

# 테스트할 3개 모델 정의
FINETUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyrDW:ckpt-step-124"
BASELINE_MINI_MODEL = "gpt-4o-mini"  # 파인튜닝 베이스 모델
BASELINE_4O_MODEL = "gpt-4o"         # 플래그십 모델

# API 호출 설정
TEMPERATURE = 0.2
CSV_FILE_PATH = "hehe.csv"

# 시스템 프롬프트
SYSTEM_PROMPT = """
당신은 ERD 설계 전문가입니다. 프로젝트 요구사항을 분석하여 완전한 데이터베이스 스키마를 생성합니다.
**핵심 원칙:**
- 백슬래시(\\) 절대 사용 금지
- erd_relationships의 모든 테이블과 외래키는 반드시 erd_tables에 존재해야 함
- 외래키 컬럼은 is_foreign_key: true 설정 필수
- 순수 JSON만 응답 (마크다운 블록 금지)
**설계 순서:**
1. 엔티티 식별 → 2. 속성/기본키 정의 → 3. 관계 분석/외래키 추가 → 4. 관계 정보 작성
**중요: 모든 관계의 테이블명과 외래키가 테이블 정의와 정확히 일치해야 합니다.**
"""

# 의미적 유사도 계산을 위한 모델
try:
    semantic_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    SEMANTIC_SIMILARITY_AVAILABLE = True
except:
    print("⚠️ 의미적 유사도 모델을 로드할 수 없습니다. 해당 메트릭은 제외됩니다.")
    SEMANTIC_SIMILARITY_AVAILABLE = False

# ========================================================================================
# 유틸리티 함수
# ========================================================================================

def safe_parse_json_string(json_str):
    """JSON 문자열을 안전하게 파싱하는 함수"""
    if pd.isna(json_str) or json_str == '':
        return None
    
    try:
        if isinstance(json_str, (dict, list)):
            return json_str
        
        if isinstance(json_str, str):
            try:
                return ast.literal_eval(json_str)
            except:
                return json.loads(json_str)
    except Exception as e:
        print(f"JSON 파싱 오류: {e}")
        return json_str

def format_data_for_prompt(data):
    """데이터를 프롬프트에 적합한 형태로 포맷팅"""
    if isinstance(data, str):
        return data
    elif isinstance(data, (dict, list)):
        return json.dumps(data, ensure_ascii=False, indent=2)
    else:
        return str(data)

def extract_text_content(data):
    """JSON 데이터에서 텍스트 내용을 추출하여 비교 가능한 형태로 변환"""
    if isinstance(data, str):
        return data.strip()
    elif isinstance(data, dict):
        text_parts = []
        for key, value in data.items():
            if isinstance(value, str):
                text_parts.append(f"{key}: {value}")
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, str):
                        text_parts.append(item)
                    elif isinstance(item, dict):
                        text_parts.append(json.dumps(item, ensure_ascii=False))
        return " ".join(text_parts)
    elif isinstance(data, list):
        text_parts = []
        for item in data:
            if isinstance(item, str):
                text_parts.append(item)
            elif isinstance(item, dict):
                text_parts.append(json.dumps(item, ensure_ascii=False))
        return " ".join(text_parts)
    else:
        return str(data)

# ========================================================================================
# 핵심 함수 정의
# ========================================================================================

def openai_chat_completion(model, user_input, project_info):
    """OpenAI Chat Completion API를 호출하는 함수"""
    try:
        formatted_project_info = format_data_for_prompt(project_info)
        formatted_user_input = format_data_for_prompt(user_input)
        
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
프로젝트 정보:
{formatted_project_info}

요구사항:
{formatted_user_input}

  **필수 JSON 형식:**
  {{
    "erd_tables": [{{
      "name": "테이블명",
      "erd_columns": [{{
        "name": "컬럼명",
        "data_type": "타입",
        "is_primary_key": true/false,
        "is_foreign_key": true/false,
        "is_nullable": true/false
      }}]
    }}],
    "erd_relationships": [{{
      "from_table": "시작테이블",
      "to_table": "끝테이블",
      "relationship_type": "one-to-many",
      "foreign_key": "외래키명",
      "constraint_name": "제약조건명"
    }}]
  }}
  **핵심 규칙:**
  1. 관계의 모든 테이블명이 erd_tables에 존재해야 함
  2. 관계의 모든 foreign_key가 해당 테이블 컬럼에 존재해야 함
  3. 외래키는 is_foreign_key: true 설정
  4. 최소 5개 테이블, 백슬래시 금지, 순수 JSON만
  위 규칙을 지켜 완전한 ERD를 생성하세요!

"""}
        ]
        
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMPERATURE,
            max_tokens=2000
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"API 호출 오류 ({model}): {e}")
        return ""

def calculate_semantic_similarity(predictions, references):
    """의미적 유사도를 계산하는 함수"""
    if not SEMANTIC_SIMILARITY_AVAILABLE:
        return 0.0
    
    try:
        pred_texts = [extract_text_content(pred) for pred in predictions]
        ref_texts = [extract_text_content(ref) for ref in references]
        
        pred_embeddings = semantic_model.encode(pred_texts)
        ref_embeddings = semantic_model.encode(ref_texts)
        
        similarities = []
        for pred_emb, ref_emb in zip(pred_embeddings, ref_embeddings):
            similarity = cosine_similarity([pred_emb], [ref_emb])[0][0]
            similarities.append(similarity)
        
        return np.mean(similarities) * 100
    except Exception as e:
        print(f"의미적 유사도 계산 오류: {e}")
        return 0.0

def calculate_metrics(predictions, references):
    """모델 성능을 평가하는 다양한 메트릭을 계산하는 함수"""
    pred_texts = [extract_text_content(pred) for pred in predictions]
    ref_texts = [extract_text_content(ref) for ref in references]
    
    valid_pairs = [(p, r) for p, r in zip(pred_texts, ref_texts) if p.strip() and r.strip()]
    
    if not valid_pairs:
        return {
            "BLEU": 0.0,
            "ROUGE-L": 0.0,
            "정확일치율(%)": 0.0,
            "의미적_유사도(%)": 0.0
        }
    
    valid_preds, valid_refs = zip(*valid_pairs)
    
    # BLEU 점수 계산
    try:
        bleu_score = corpus_bleu(list(valid_preds), [list(valid_refs)]).score
    except:
        bleu_score = 0.0
    
    # ROUGE-L 점수 계산
    try:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        rouge_scores = [scorer.score(ref, pred)['rougeL'].fmeasure 
                       for ref, pred in zip(valid_refs, valid_preds)]
        rouge_l_score = np.mean(rouge_scores) * 100
    except:
        rouge_l_score = 0.0
    
    # 정확 일치율 계산
    exact_matches = sum(1 for p, r in zip(valid_preds, valid_refs) 
                       if p.strip() == r.strip())
    exact_match_rate = (exact_matches / len(valid_pairs)) * 100
    
    # 의미적 유사도 계산
    semantic_similarity = calculate_semantic_similarity(predictions, references)
    
    return {
        "BLEU": round(bleu_score, 2),
        "ROUGE-L": round(rouge_l_score, 2),
        "정확일치율(%)": round(exact_match_rate, 2),
        "의미적_유사도(%)": round(semantic_similarity, 2)
    }

def run_three_model_evaluation():
    """3개 모델 비교 평가 실행 함수"""
    print("📊 3개 모델 성능 비교: 파인튜닝 vs GPT-4o-mini vs GPT-4o")
    print("=" * 80)
    
    # ========================================================================================
    # 1. 데이터 로드 및 검증
    # ========================================================================================
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"✅ 데이터 로드 완료: {len(df)}개 샘플")
        print(f"📋 CSV 컬럼: {list(df.columns)}")
            
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return
    
    # 데이터 전처리
    print("\n🔄 데이터 전처리 중...")
    for col in ['user_input', 'project_info', 'total_requirements', 'ERD_data']:
        if col in df.columns:
            df[col] = df[col].apply(safe_parse_json_string)
    
    # ========================================================================================
    # 2. 3개 모델 추론 실행
    # ========================================================================================
    
    # 테스트할 샘플 수 제한
    test_samples = min(5, len(df))
    test_df = df.head(test_samples).copy()
    
    print(f"\n🔄 {test_samples}개 샘플에 대해 3개 모델 추론 시작...")
    
    # 3개 모델의 결과를 저장할 딕셔너리
    results = {
        FINETUNED_MODEL: [],
        BASELINE_MINI_MODEL: [],
        BASELINE_4O_MODEL: []
    }
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="모델 추론 진행"):
        user_input = row['user_input']
        project_info = row['project_info']
        
        print(f"\n📝 샘플 {idx+1} 처리 중...")
        
        # 1. 파인튜닝 모델 추론
        print(f"   🎯 파인튜닝 모델 추론...")
        ft_result = openai_chat_completion(FINETUNED_MODEL, user_input, project_info)
        results[FINETUNED_MODEL].append(ft_result)
        
        # 2. GPT-4o-mini 원본 추론
        print(f"   🤖 GPT-4o-mini 추론...")
        mini_result = openai_chat_completion(BASELINE_MINI_MODEL, user_input, project_info)
        results[BASELINE_MINI_MODEL].append(mini_result)
        
        # 3. GPT-4o 추론
        print(f"   🔥 GPT-4o 추론...")
        gpt4o_result = openai_chat_completion(BASELINE_4O_MODEL, user_input, project_info)
        results[BASELINE_4O_MODEL].append(gpt4o_result)
        
        time.sleep(1.0)  # API 레이트 리미트 방지
    
    # ========================================================================================
    # 3. 성능 메트릭 계산 및 비교
    # ========================================================================================
    
    print("\n📈 3개 모델 성능 평가 결과")
    print("=" * 80)
    
    # 정답 데이터
    references = test_df['ERD_data'].tolist()
    
    # 모델별 성능 계산
    final_results = {}
    model_names = {
        FINETUNED_MODEL: "파인튜닝_모델",
        BASELINE_MINI_MODEL: "GPT-4o-mini", 
        BASELINE_4O_MODEL: "GPT-4o"
    }
    
    for model_id, predictions in results.items():
        model_name = model_names[model_id]
        
        print(f"\n🔍 {model_name} 출력 확인:")
        for i, pred in enumerate(predictions[:2]):
            print(f"  출력 {i+1}: {str(pred)[:150]}...")
        
        metrics = calculate_metrics(predictions, references)
        
        print(f"\n🤖 {model_name} 성능:")
        for metric_name, value in metrics.items():
            print(f"   {metric_name}: {value}")
        
        final_results[model_name] = metrics
    
    # ========================================================================================
    # 4. 종합 비교 분석
    # ========================================================================================
    
    print(f"\n📊 종합 성능 비교 분석")
    print("=" * 80)
    
    # 성능 비교 표 출력
    metrics_list = ["BLEU", "ROUGE-L", "의미적_유사도(%)"]
    
    print(f"{'메트릭':<15} {'파인튜닝':<12} {'4o-mini':<12} {'GPT-4o':<12} {'vs mini':<12} {'vs 4o':<12}")
    print("-" * 80)
    
    for metric in metrics_list:
        ft_score = final_results["파인튜닝_모델"][metric]
        mini_score = final_results["GPT-4o-mini"][metric]
        gpt4o_score = final_results["GPT-4o"][metric]
        
        vs_mini = ft_score - mini_score
        vs_4o = ft_score - gpt4o_score
        
        print(f"{metric:<15} {ft_score:<12.2f} {mini_score:<12.2f} {gpt4o_score:<12.2f} {vs_mini:<+12.2f} {vs_4o:<+12.2f}")
    
    # ========================================================================================
    # 5. 파인튜닝 효과 분석
    # ========================================================================================
    
    print(f"\n🎯 파인튜닝 효과 분석")
    print("=" * 80)
    
    ft_metrics = final_results["파인튜닝_모델"]
    mini_metrics = final_results["GPT-4o-mini"]
    gpt4o_metrics = final_results["GPT-4o"]
    
    # 베이스 모델(4o-mini) 대비 개선도
    print("📈 베이스 모델(GPT-4o-mini) 대비 파인튜닝 개선도:")
    mini_improvements = 0
    for metric in metrics_list:
        diff = ft_metrics[metric] - mini_metrics[metric]
        improvement_rate = (diff / mini_metrics[metric]) * 100 if mini_metrics[metric] != 0 else 0
        print(f"  {metric}: {diff:+.2f} ({improvement_rate:+.1f}%)")
        if diff > 0:
            mini_improvements += 1
    
    # 플래그십 모델(GPT-4o) 대비 성능
    print(f"\n🔥 플래그십 모델(GPT-4o) 대비 파인튜닝 성능:")
    gpt4o_wins = 0
    for metric in metrics_list:
        diff = ft_metrics[metric] - gpt4o_metrics[metric]
        print(f"  {metric}: {diff:+.2f} ({'우수' if diff > 0 else '열세'})")
        if diff > 0:
            gpt4o_wins += 1
    
    # ========================================================================================
    # 6. 결론 및 권장사항
    # ========================================================================================
    
    print(f"\n🏆 최종 결론")
    print("=" * 80)
    
    if mini_improvements >= 2:
        print("✅ 파인튜닝이 베이스 모델 대비 명확한 성능 향상을 보여줍니다!")
    else:
        print("⚠️ 파인튜닝 효과가 제한적입니다.")
    
    if gpt4o_wins >= 2:
        print("🔥 파인튜닝 모델이 플래그십 모델도 능가하는 놀라운 성과입니다!")
    elif gpt4o_wins >= 1:
        print("👍 파인튜닝 모델이 일부 지표에서 플래그십 모델과 경쟁합니다!")
    else:
        print("📝 플래그십 모델 대비로는 아직 개선 여지가 있습니다.")
    
    # 비용 효율성 분석
    print(f"\n💰 비용 효율성 분석:")
    print(f"  파인튜닝 모델 비용: GPT-4o의 ~10% (훨씬 저렴)")
    print(f"  추론 속도: GPT-4o보다 빠름")
    if gpt4o_wins >= 1:
        print(f"  성능: 일부 지표에서 GPT-4o 수준 또는 그 이상")
        print(f"  → 🎯 매우 높은 ROI!")
    
    # ========================================================================================
    # 7. 결과 저장
    # ========================================================================================
    
    try:
        with open("3모델_성능비교_결과.json", "w", encoding="utf-8") as f:
            json.dump(final_results, f, ensure_ascii=False, indent=2)
        print(f"\n💾 결과가 '3모델_성능비교_결과.json' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"❌ 결과 저장 실패: {e}")

# ========================================================================================
# 메인 실행 부분
# ========================================================================================

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY 환경변수를 설정해주세요.")
        exit(1)
    
    print("🚀 3개 모델 성능 비교 테스트를 시작합니다...")
    print(f"📁 테스트 데이터: {CSV_FILE_PATH}")
    print(f"🎯 파인튜닝 모델: {FINETUNED_MODEL}")
    print(f"🤖 GPT-4o-mini: {BASELINE_MINI_MODEL}")
    print(f"🔥 GPT-4o: {BASELINE_4O_MODEL}")
    print(f"🌡️ 온도 설정: {TEMPERATURE}")
    
    run_three_model_evaluation()
    
    print("\n✅ 3개 모델 성능 비교 테스트가 완료되었습니다!")

W0627 16:55:04.859000 24772 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
c:\Users\mir96\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


🚀 3개 모델 성능 비교 테스트를 시작합니다...
📁 테스트 데이터: hehe.csv
🎯 파인튜닝 모델: ft:gpt-4o-mini-2024-07-18:test:pja-erd-finetuning-model:BmOgyrDW:ckpt-step-124
🤖 GPT-4o-mini: gpt-4o-mini
🔥 GPT-4o: gpt-4o
🌡️ 온도 설정: 0.2
📊 3개 모델 성능 비교: 파인튜닝 vs GPT-4o-mini vs GPT-4o
✅ 데이터 로드 완료: 138개 샘플
📋 CSV 컬럼: ['user_input', 'total_requirements', 'project_info', 'ERD_data']

🔄 데이터 전처리 중...

🔄 5개 샘플에 대해 3개 모델 추론 시작...


모델 추론 진행:   0%|          | 0/5 [00:00<?, ?it/s]


📝 샘플 1 처리 중...
   🎯 파인튜닝 모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행:  20%|██        | 1/5 [01:02<04:09, 62.47s/it]

...


📝 샘플 4 처리 중...
   🎯 파인튜닝 모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행:  80%|████████  | 4/5 [03:42<00:53, 53.48s/it]


📝 샘플 5 처리 중...
   🎯 파인튜닝 모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행: 100%|██████████| 5/5 [04:33<00:00, 54.65s/it]



📈 3개 모델 성능 평가 결과

🔍 파인튜닝_모델 출력 확인:
  출력 1: {
  "erd_tables": [
    {
      "name": "사용자",
      "erd_columns": [
        {
          "name": "사용자ID",
          "data_type": "INTEGER",
         ...
  출력 2: {
  "erd_tables": [
    {
      "name": "사용자",
      "erd_columns": [
        {
          "name": "사용자ID",
          "data_type": "ObjectId",
        ...

🤖 파인튜닝_모델 성능:
   BLEU: 75.17
   ROUGE-L: 79.73
   정확일치율(%): 0.0
   의미적_유사도(%): 94.29

🔍 GPT-4o-mini 출력 확인:
  출력 1: {
  "erd_tables": [
    {
      "name": "users",
      "erd_columns": [
        {
          "name": "user_id",
          "data_type": "SERIAL",
      ...
  출력 2: {
  "erd_tables": [
    {
      "name": "Users",
      "erd_columns": [
        {
          "name": "user_id",
          "data_type": "ObjectId",
    ...

🤖 GPT-4o-mini 성능:
   BLEU: 84.07
   ROUGE-L: 84.61
   정확일치율(%): 0.0
   의미적_유사도(%): 95.98

🔍 GPT-4o 출력 확인:
  출력 1: {
  "erd_tables": [
    {
      "name": "users",
      "erd_columns": [
        {
          "

In [5]:
df = pd.read_csv("모델_출력_비교_개선.csv")

In [6]:
df.head()

,user_input,total_requirements,project_info,ERD_data,파인튜닝_모델_출력,GPT4o_출력
0,"[{'projectName': '스터디 그룹 성과 분석 도구', 'projectTa...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '스터디 그룹 성과 분석 도구', ...","{'erd_tables': [{'name': 'users', 'erd_columns...","아래는 ""스터디 그룹 성과 분석 도구"" 프로젝트에 대한 ERD(Entity Rela...","아래는 ""스터디 그룹 성과 분석 도구"" 프로젝트를 위한 ERD 데이터입니다. 이 데..."
1,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '기억의 다리', 'category...","{'erd_tables': [{'name': 'Users', 'erd_columns...","아래는 ""기억의 다리"" 프로젝트를 위한 ERD(Entity Relationship ...","```json\n{\n ""entities"": [\n {\n ""nam..."
2,"[{'projectName': '장애인 친화 대중교통 안내 앱', 'projectT...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '장애인 친화 대중교통 안내 앱',...","{'erd_tables': [{'name': 'Users', 'erd_columns...","아래는 ""장애인 친화 대중교통 안내 앱""의 ERD(Entity Relationshi...","```json\n{\n ""entities"": [\n {\n ""nam..."
3,"[{'projectName': '가상 피팅룸 서비스', 'projectTarget'...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '가상 피팅룸 서비스', 'cate...","{'erd_tables': [{'name': '사용자', 'erd_columns':...","아래는 ""가상 피팅룸 서비스""를 위한 ERD(Entity Relationship D...","```json\n{\n ""entities"": [\n {\n ""nam..."
4,"[{'projectName': '스마트 건강 관리 비서', 'projectTarge...","[{'requirementType': 'FUNCTIONAL', 'content': ...","{'project_info': {'title': '스마트 건강 관리 비서', 'ca...","{'erd_tables': [{'name': 'Users', 'erd_columns...","아래는 ""스마트 건강 관리 비서"" 프로젝트를 위한 ERD(Entity Relatio...","```json\n{\n ""entities"": [\n {\n ""nam..."


In [9]:
"""
ERD 특화 3모델 성능 비교: 구조적 품질과 정확성 중심 평가
JSON 구조, 스키마 정확성, ERD 설계 품질을 직접 측정하는 전문 평가 시스템
"""

import os
import json
import time
import pandas as pd
from tqdm import tqdm
import openai
import numpy as np
import ast
from collections import defaultdict
import re
from typing import Dict, List, Tuple, Any

# ========================================================================================
# 전역 설정 및 상수 정의
# ========================================================================================

# 테스트할 3개 모델 정의
FINETUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:test:pja-api-finetuning-model:BmOdcDUE"
BASELINE_MINI_MODEL = "gpt-4o-mini"
BASELINE_4O_MODEL = "gpt-4o"

# API 호출 설정
TEMPERATURE = 0.2
CSV_FILE_PATH = "hehe.csv"

# 시스템 프롬프트
SYSTEM_PROMPT = """
당신은 ERD 설계 전문가입니다. 프로젝트 요구사항을 분석하여 완전한 데이터베이스 스키마를 생성합니다.
**핵심 원칙:**
- 백슬래시(\\) 절대 사용 금지
- erd_relationships의 모든 테이블과 외래키는 반드시 erd_tables에 존재해야 함
- 외래키 컬럼은 is_foreign_key: true 설정 필수
- 순수 JSON만 응답 (마크다운 블록 금지)
**설계 순서:**
1. 엔티티 식별 → 2. 속성/기본키 정의 → 3. 관계 분석/외래키 추가 → 4. 관계 정보 작성
**중요: 모든 관계의 테이블명과 외래키가 테이블 정의와 정확히 일치해야 합니다.**
"""

# ========================================================================================
# ERD 특화 평가 함수들
# ========================================================================================

def safe_parse_json(json_str: str) -> Tuple[Dict, bool]:
    """JSON 파싱을 시도하고 성공 여부를 반환"""
    if pd.isna(json_str) or json_str == '':
        return {}, False
    
    try:
        if isinstance(json_str, (dict, list)):
            return json_str, True
        
        if isinstance(json_str, str):
            # 마크다운 코드 블록 제거
            json_str = re.sub(r'```json\s*|\s*```', '', json_str.strip())
            json_str = json_str.strip()
            
            try:
                return json.loads(json_str), True
            except:
                return ast.literal_eval(json_str), True
    except Exception as e:
        return {}, False

def validate_erd_structure(erd_data: Dict) -> Dict[str, Any]:
    """ERD JSON 구조의 유효성을 검사"""
    validation_result = {
        'is_valid_json': True,
        'has_erd_tables': False,
        'has_erd_relationships': False,
        'table_count': 0,
        'relationship_count': 0,
        'column_count': 0,
        'foreign_key_count': 0,
        'primary_key_count': 0,
        'structural_issues': []
    }
    
    if not isinstance(erd_data, dict):
        validation_result['is_valid_json'] = False
        validation_result['structural_issues'].append("Invalid JSON structure")
        return validation_result
    
    # 필수 키 존재 확인
    if 'erd_tables' in erd_data:
        validation_result['has_erd_tables'] = True
        tables = erd_data['erd_tables']
        if isinstance(tables, list):
            validation_result['table_count'] = len(tables)
            
            # 테이블별 컬럼 분석
            for table in tables:
                if isinstance(table, dict) and 'erd_columns' in table:
                    columns = table['erd_columns']
                    if isinstance(columns, list):
                        validation_result['column_count'] += len(columns)
                        
                        for col in columns:
                            if isinstance(col, dict):
                                if col.get('is_foreign_key', False):
                                    validation_result['foreign_key_count'] += 1
                                if col.get('is_primary_key', False):
                                    validation_result['primary_key_count'] += 1
        else:
            validation_result['structural_issues'].append("erd_tables is not a list")
    else:
        validation_result['structural_issues'].append("Missing erd_tables")
    
    if 'erd_relationships' in erd_data:
        validation_result['has_erd_relationships'] = True
        relationships = erd_data['erd_relationships']
        if isinstance(relationships, list):
            validation_result['relationship_count'] = len(relationships)
        else:
            validation_result['structural_issues'].append("erd_relationships is not a list")
    else:
        validation_result['structural_issues'].append("Missing erd_relationships")
    
    return validation_result

def calculate_schema_consistency(erd_data: Dict) -> Dict[str, float]:
    """ERD 스키마의 일관성을 계산"""
    consistency_metrics = {
        'foreign_key_consistency': 0.0,  # 외래키가 실제 테이블에 존재하는가
        'relationship_consistency': 0.0,  # 관계의 테이블들이 실제 존재하는가
        'data_type_consistency': 0.0,    # 데이터 타입이 일관되게 정의되었는가
        'naming_consistency': 0.0        # 네이밍 규칙이 일관되는가
    }
    
    if not isinstance(erd_data, dict):
        return consistency_metrics
    
    tables = erd_data.get('erd_tables', [])
    relationships = erd_data.get('erd_relationships', [])
    
    if not tables:
        return consistency_metrics
    
    # 테이블명과 컬럼 정보 수집
    table_names = set()
    table_columns = {}
    all_columns = []
    
    for table in tables:
        if isinstance(table, dict) and 'name' in table:
            table_name = table['name']
            table_names.add(table_name)
            
            columns = table.get('erd_columns', [])
            table_columns[table_name] = {}
            
            for col in columns:
                if isinstance(col, dict) and 'name' in col:
                    col_name = col['name']
                    table_columns[table_name][col_name] = col
                    all_columns.append(col)
    
    # 1. 외래키 일관성 검사
    if all_columns:
        foreign_keys = [col for col in all_columns if col.get('is_foreign_key', False)]
        if foreign_keys:
            valid_fk_count = 0
            for fk in foreign_keys:
                # 외래키가 참조하는 테이블이 존재하는지 간접적으로 확인
                # (실제 참조 테이블 정보가 없으므로 외래키 명명 규칙으로 추정)
                fk_name = fk.get('name', '')
                if '_id' in fk_name or 'id' in fk_name:
                    valid_fk_count += 1
            consistency_metrics['foreign_key_consistency'] = valid_fk_count / len(foreign_keys) * 100
        else:
            consistency_metrics['foreign_key_consistency'] = 100.0  # 외래키가 없으면 만점
    
    # 2. 관계 일관성 검사
    if relationships and table_names:
        valid_relationship_count = 0
        for rel in relationships:
            if isinstance(rel, dict):
                from_table = rel.get('from_table', '')
                to_table = rel.get('to_table', '')
                foreign_key = rel.get('foreign_key', '')
                
                # 테이블 존재 여부 확인
                tables_exist = from_table in table_names and to_table in table_names
                
                # 외래키가 실제 컬럼에 존재하는지 확인
                fk_exists = False
                if from_table in table_columns:
                    fk_exists = foreign_key in table_columns[from_table]
                
                if tables_exist and fk_exists:
                    valid_relationship_count += 1
        
        consistency_metrics['relationship_consistency'] = valid_relationship_count / len(relationships) * 100
    else:
        consistency_metrics['relationship_consistency'] = 100.0 if not relationships else 0.0
    
    # 3. 데이터 타입 일관성 (공통 타입들이 일관되게 사용되는가)
    if all_columns:
        data_types = [col.get('data_type', '').lower() for col in all_columns if col.get('data_type')]
        if data_types:
            # 표준 데이터 타입과의 일치도 측정
            standard_types = {'int', 'varchar', 'text', 'date', 'datetime', 'boolean', 'decimal', 'float'}
            standard_type_count = sum(1 for dt in data_types if any(st in dt for st in standard_types))
            consistency_metrics['data_type_consistency'] = standard_type_count / len(data_types) * 100
        else:
            consistency_metrics['data_type_consistency'] = 0.0
    
    # 4. 네이밍 일관성 (snake_case 등 일관된 명명 규칙)
    all_names = list(table_names)
    for cols in table_columns.values():
        all_names.extend(cols.keys())
    
    if all_names:
        snake_case_count = sum(1 for name in all_names if re.match(r'^[a-z][a-z0-9_]*$', name))
        consistency_metrics['naming_consistency'] = snake_case_count / len(all_names) * 100
    
    return consistency_metrics

def compare_erd_structures(pred_erd: Dict, ref_erd: Dict) -> Dict[str, float]:
    """두 ERD 구조를 비교하여 유사도를 계산"""
    comparison_metrics = {
        'table_name_similarity': 0.0,      # 테이블명 일치도
        'column_structure_similarity': 0.0, # 컬럼 구조 유사도
        'relationship_similarity': 0.0,     # 관계 구조 유사도
        'overall_structure_similarity': 0.0 # 전체 구조 유사도
    }
    
    # 테이블명 비교
    pred_tables = {table.get('name', '') for table in pred_erd.get('erd_tables', []) if isinstance(table, dict)}
    ref_tables = {table.get('name', '') for table in ref_erd.get('erd_tables', []) if isinstance(table, dict)}
    
    if ref_tables:
        table_intersection = pred_tables.intersection(ref_tables)
        comparison_metrics['table_name_similarity'] = len(table_intersection) / len(ref_tables) * 100
    
    # 컬럼 구조 비교
    pred_columns = set()
    ref_columns = set()
    
    for table in pred_erd.get('erd_tables', []):
        if isinstance(table, dict):
            table_name = table.get('name', '')
            for col in table.get('erd_columns', []):
                if isinstance(col, dict):
                    pred_columns.add(f"{table_name}.{col.get('name', '')}")
    
    for table in ref_erd.get('erd_tables', []):
        if isinstance(table, dict):
            table_name = table.get('name', '')
            for col in table.get('erd_columns', []):
                if isinstance(col, dict):
                    ref_columns.add(f"{table_name}.{col.get('name', '')}")
    
    if ref_columns:
        column_intersection = pred_columns.intersection(ref_columns)
        comparison_metrics['column_structure_similarity'] = len(column_intersection) / len(ref_columns) * 100
    
    # 관계 비교
    pred_relationships = set()
    ref_relationships = set()
    
    for rel in pred_erd.get('erd_relationships', []):
        if isinstance(rel, dict):
            rel_key = f"{rel.get('from_table', '')}->{rel.get('to_table', '')}"
            pred_relationships.add(rel_key)
    
    for rel in ref_erd.get('erd_relationships', []):
        if isinstance(rel, dict):
            rel_key = f"{rel.get('from_table', '')}->{rel.get('to_table', '')}"
            ref_relationships.add(rel_key)
    
    if ref_relationships:
        rel_intersection = pred_relationships.intersection(ref_relationships)
        comparison_metrics['relationship_similarity'] = len(rel_intersection) / len(ref_relationships) * 100
    
    # 전체 구조 유사도 (가중 평균)
    comparison_metrics['overall_structure_similarity'] = (
        comparison_metrics['table_name_similarity'] * 0.4 +
        comparison_metrics['column_structure_similarity'] * 0.4 +
        comparison_metrics['relationship_similarity'] * 0.2
    )
    
    return comparison_metrics

def calculate_erd_quality_score(erd_data: Dict) -> Dict[str, float]:
    """ERD 설계 품질 점수를 계산"""
    quality_metrics = {
        'completeness_score': 0.0,    # 완성도 (필수 요소 포함 여부)
        'complexity_score': 0.0,      # 복잡도 (적절한 테이블/관계 수)
        'normalization_score': 0.0,   # 정규화 점수 (기본키/외래키 적절성)
        'design_quality_score': 0.0   # 설계 품질 종합 점수
    }
    
    validation = validate_erd_structure(erd_data)
    
    # 1. 완성도 점수
    completeness_factors = []
    completeness_factors.append(100 if validation['has_erd_tables'] else 0)
    completeness_factors.append(100 if validation['has_erd_relationships'] else 0)
    completeness_factors.append(100 if validation['table_count'] >= 3 else validation['table_count'] * 33.3)
    completeness_factors.append(100 if validation['primary_key_count'] >= validation['table_count'] else 
                               (validation['primary_key_count'] / max(validation['table_count'], 1)) * 100)
    
    quality_metrics['completeness_score'] = sum(completeness_factors) / len(completeness_factors)
    
    # 2. 복잡도 점수 (적절한 복잡도인가)
    table_count = validation['table_count']
    relationship_count = validation['relationship_count']
    
    # 3-10개 테이블이 적절, 관계는 테이블 수와 비슷하거나 적게
    if 3 <= table_count <= 10:
        complexity_table_score = 100
    elif table_count < 3:
        complexity_table_score = table_count * 33.3
    else:
        complexity_table_score = max(0, 100 - (table_count - 10) * 5)
    
    if relationship_count <= table_count:
        complexity_rel_score = 100
    else:
        complexity_rel_score = max(0, 100 - (relationship_count - table_count) * 10)
    
    quality_metrics['complexity_score'] = (complexity_table_score + complexity_rel_score) / 2
    
    # 3. 정규화 점수
    normalization_factors = []
    
    # 기본키 존재율
    if validation['table_count'] > 0:
        pk_ratio = validation['primary_key_count'] / validation['table_count']
        normalization_factors.append(min(pk_ratio * 100, 100))
    
    # 외래키 활용도
    if validation['relationship_count'] > 0:
        fk_ratio = validation['foreign_key_count'] / validation['relationship_count']
        normalization_factors.append(min(fk_ratio * 100, 100))
    else:
        normalization_factors.append(100)  # 관계가 없으면 만점
    
    if normalization_factors:
        quality_metrics['normalization_score'] = sum(normalization_factors) / len(normalization_factors)
    
    # 4. 설계 품질 종합 점수
    quality_metrics['design_quality_score'] = (
        quality_metrics['completeness_score'] * 0.4 +
        quality_metrics['complexity_score'] * 0.3 +
        quality_metrics['normalization_score'] * 0.3
    )
    
    return quality_metrics

# ========================================================================================
# API 호출 및 메인 평가 함수
# ========================================================================================

def format_data_for_prompt(data):
    """데이터를 프롬프트에 적합한 형태로 포맷팅"""
    if isinstance(data, str):
        return data
    elif isinstance(data, (dict, list)):
        return json.dumps(data, ensure_ascii=False, indent=2)
    else:
        return str(data)

def openai_chat_completion(model, user_input, project_info):
    """OpenAI Chat Completion API를 호출하는 함수"""
    try:
        formatted_project_info = format_data_for_prompt(project_info)
        formatted_user_input = format_data_for_prompt(user_input)
        
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
프로젝트 정보:
{formatted_project_info}

요구사항:
{formatted_user_input}

**필수 JSON 형식:**
{{
  "erd_tables": [{{
    "name": "테이블명",
    "erd_columns": [{{
      "name": "컬럼명",
      "data_type": "타입",
      "is_primary_key": true/false,
      "is_foreign_key": true/false,
      "is_nullable": true/false
    }}]
  }}],
  "erd_relationships": [{{
    "from_table": "시작테이블",
    "to_table": "끝테이블",
    "relationship_type": "one-to-many",
    "foreign_key": "외래키명",
    "constraint_name": "제약조건명"
  }}]
}}
**핵심 규칙:**
1. 관계의 모든 테이블명이 erd_tables에 존재해야 함
2. 관계의 모든 foreign_key가 해당 테이블 컬럼에 존재해야 함
3. 외래키는 is_foreign_key: true 설정
4. 최소 5개 테이블, 백슬래시 금지, 순수 JSON만
위 규칙을 지켜 완전한 ERD를 생성하세요!
"""}
        ]
        
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            temperature=TEMPERATURE,
            max_tokens=2000
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"API 호출 오류 ({model}): {e}")
        return ""

def run_erd_specialized_evaluation():
    """ERD 특화 3모델 비교 평가 실행"""
    print("🎯 ERD 특화 3모델 성능 비교: 구조적 품질과 정확성 중심 평가")
    print("=" * 80)
    
    # ========================================================================================
    # 1. 데이터 로드 및 검증
    # ========================================================================================
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"✅ 데이터 로드 완료: {len(df)}개 샘플")
        print(f"📋 CSV 컬럼: {list(df.columns)}")
            
    except Exception as e:
        print(f"❌ CSV 파일 로드 실패: {e}")
        return
    
    # 데이터 전처리
    print("\n🔄 데이터 전처리 중...")
    for col in ['user_input', 'project_info', 'total_requirements', 'ERD_data']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: safe_parse_json(str(x))[0] if pd.notna(x) else {})
    
    # ========================================================================================
    # 2. 테스트 샘플 설정 (더 많은 샘플로 확장)
    # ========================================================================================
    
    test_samples = min(5, len(df))  # 20개 샘플로 확장
    test_df = df.head(test_samples).copy()
    
    print(f"\n🔄 {test_samples}개 샘플에 대해 3개 모델 ERD 생성 시작...")
    
    # ========================================================================================
    # 3. 3개 모델 추론 실행
    # ========================================================================================
    
    results = {
        'finetuned': {'predictions': [], 'model_name': '파인튜닝_모델'},
        'mini': {'predictions': [], 'model_name': 'GPT-4o-mini'},
        'gpt4o': {'predictions': [], 'model_name': 'GPT-4o'}
    }
    
    models = [
        (FINETUNED_MODEL, 'finetuned', '🎯'),
        (BASELINE_MINI_MODEL, 'mini', '🤖'),
        (BASELINE_4O_MODEL, 'gpt4o', '🔥')
    ]
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="모델 추론 진행"):
        user_input = row['user_input']
        project_info = row['project_info']
        
        print(f"\n📝 샘플 {idx+1} 처리 중...")
        
        for model_id, result_key, emoji in models:
            print(f"   {emoji} {results[result_key]['model_name']} 추론...")
            
            prediction = openai_chat_completion(model_id, user_input, project_info)
            results[result_key]['predictions'].append(prediction)
            
            time.sleep(0.5)  # API 레이트 리미트 방지
    
    # ========================================================================================
    # 4. ERD 특화 평가 메트릭 계산
    # ========================================================================================
    
    print("\n📊 ERD 특화 평가 결과 분석")
    print("=" * 80)
    
    references = test_df['ERD_data'].tolist()
    evaluation_results = {}
    
    for result_key, result_data in results.items():
        model_name = result_data['model_name']
        predictions = result_data['predictions']
        
        print(f"\n🔍 {model_name} 분석 중...")
        
        # JSON 파싱 성공률 계산
        parsed_predictions = []
        json_success_count = 0
        
        for pred in predictions:
            parsed_pred, is_valid = safe_parse_json(pred)
            parsed_predictions.append(parsed_pred)
            if is_valid:
                json_success_count += 1
        
        json_success_rate = (json_success_count / len(predictions)) * 100
        
        # ERD 구조 검증
        structure_validations = [validate_erd_structure(pred) for pred in parsed_predictions]
        
        # 스키마 일관성 계산
        consistency_scores = [calculate_schema_consistency(pred) for pred in parsed_predictions]
        
        # ERD 품질 점수 계산
        quality_scores = [calculate_erd_quality_score(pred) for pred in parsed_predictions]
        
        # 참조 데이터와 비교
        comparison_scores = []
        for pred, ref in zip(parsed_predictions, references):
            if isinstance(ref, dict) and ref:
                comparison = compare_erd_structures(pred, ref)
                comparison_scores.append(comparison)
            else:
                comparison_scores.append({
                    'table_name_similarity': 0.0,
                    'column_structure_similarity': 0.0,
                    'relationship_similarity': 0.0,
                    'overall_structure_similarity': 0.0
                })
        
        # 평균 계산
        avg_metrics = {
            'JSON_파싱_성공률(%)': json_success_rate,
            '평균_테이블_수': np.mean([v['table_count'] for v in structure_validations]),
            '평균_관계_수': np.mean([v['relationship_count'] for v in structure_validations]),
            '평균_컬럼_수': np.mean([v['column_count'] for v in structure_validations]),
            '외래키_일관성(%)': np.mean([c['foreign_key_consistency'] for c in consistency_scores]),
            '관계_일관성(%)': np.mean([c['relationship_consistency'] for c in consistency_scores]),
            '네이밍_일관성(%)': np.mean([c['naming_consistency'] for c in consistency_scores]),
            '완성도_점수(%)': np.mean([q['completeness_score'] for q in quality_scores]),
            '복잡도_점수(%)': np.mean([q['complexity_score'] for q in quality_scores]),
            '정규화_점수(%)': np.mean([q['normalization_score'] for q in quality_scores]),
            '설계품질_점수(%)': np.mean([q['design_quality_score'] for q in quality_scores]),
            '테이블명_일치도(%)': np.mean([c['table_name_similarity'] for c in comparison_scores]),
            '컬럼구조_일치도(%)': np.mean([c['column_structure_similarity'] for c in comparison_scores]),
            '관계구조_일치도(%)': np.mean([c['relationship_similarity'] for c in comparison_scores]),
            '전체구조_일치도(%)': np.mean([c['overall_structure_similarity'] for c in comparison_scores])
        }
        
        evaluation_results[model_name] = avg_metrics
        
        print(f"   JSON 파싱 성공률: {json_success_rate:.1f}%")
        print(f"   평균 테이블 수: {avg_metrics['평균_테이블_수']:.1f}")
        print(f"   설계 품질 점수: {avg_metrics['설계품질_점수(%)']:.1f}%")
        print(f"   전체 구조 일치도: {avg_metrics['전체구조_일치도(%)']:.1f}%")
    
    # ========================================================================================
    # 5. 종합 비교 분석
    # ========================================================================================
    
    print(f"\n📈 ERD 특화 성능 비교 분석")
    print("=" * 100)
    
    # 핵심 메트릭들
    key_metrics = [
        'JSON_파싱_성공률(%)',
        '설계품질_점수(%)',
        '전체구조_일치도(%)',
        '관계_일관성(%)',
        '완성도_점수(%)'
    ]
    
    print(f"{'메트릭':<20} {'파인튜닝':<12} {'4o-mini':<12} {'GPT-4o':<12} {'vs mini':<12} {'vs 4o':<12}")
    print("-" * 100)
    
    ft_results = evaluation_results['파인튜닝_모델']
    mini_results = evaluation_results['GPT-4o-mini']
    gpt4o_results = evaluation_results['GPT-4o']
    
    for metric in key_metrics:
        ft_score = ft_results[metric]
        mini_score = mini_results[metric]
        gpt4o_score = gpt4o_results[metric]
        
        vs_mini = ft_score - mini_score
        vs_4o = ft_score - gpt4o_score
        
        print(f"{metric:<20} {ft_score:<12.1f} {mini_score:<12.1f} {gpt4o_score:<12.1f} {vs_mini:<+12.1f} {vs_4o:<+12.1f}")
    
    # ========================================================================================
    # 6. 세부 분석 결과
    # ========================================================================================
    
    print(f"\n🔍 세부 ERD 품질 분석")
    print("=" * 80)
    
    # 구조적 품질 분석
    print("\n📊 구조적 품질 분석:")
    structure_metrics = ['평균_테이블_수', '평균_관계_수', '평균_컬럼_수']
    
    for metric in structure_metrics:
        ft_val = ft_results[metric]
        mini_val = mini_results[metric]
        gpt4o_val = gpt4o_results[metric]
        print(f"  {metric}: 파인튜닝={ft_val:.1f}, 4o-mini={mini_val:.1f}, GPT-4o={gpt4o_val:.1f}")
    
    # 일관성 분석
    print("\n🔗 스키마 일관성 분석:")
    consistency_metrics = ['외래키_일관성(%)', '관계_일관성(%)', '네이밍_일관성(%)']
    
    for metric in consistency_metrics:
        ft_val = ft_results[metric]
        mini_val = mini_results[metric]
        gpt4o_val = gpt4o_results[metric]
        print(f"  {metric}: 파인튜닝={ft_val:.1f}%, 4o-mini={mini_val:.1f}%, GPT-4o={gpt4o_val:.1f}%")
    
    # 정확성 분석
    print("\n🎯 구조 정확성 분석:")
    accuracy_metrics = ['테이블명_일치도(%)', '컬럼구조_일치도(%)', '관계구조_일치도(%)']
    
    for metric in accuracy_metrics:
        ft_val = ft_results[metric]
        mini_val = mini_results[metric]
        gpt4o_val = gpt4o_results[metric]
        print(f"  {metric}: 파인튜닝={ft_val:.1f}%, 4o-mini={mini_val:.1f}%, GPT-4o={gpt4o_val:.1f}%")
    
    # ========================================================================================
    # 7. 파인튜닝 효과 분석
    # ========================================================================================
    
    print(f"\n🎯 파인튜닝 효과 심층 분석")
    print("=" * 80)
    
    # 베이스 모델 대비 개선도
    print("📈 베이스 모델(GPT-4o-mini) 대비 파인튜닝 개선도:")
    mini_improvements = 0
    total_improvement = 0
    
    for metric in key_metrics:
        ft_score = ft_results[metric]
        mini_score = mini_results[metric]
        diff = ft_score - mini_score
        improvement_rate = (diff / mini_score) * 100 if mini_score != 0 else 0
        
        print(f"  {metric}: {diff:+.1f} ({improvement_rate:+.1f}%)")
        
        if diff > 0:
            mini_improvements += 1
        total_improvement += improvement_rate
    
    avg_improvement = total_improvement / len(key_metrics)
    
    # 플래그십 모델 대비 성능
    print(f"\n🔥 플래그십 모델(GPT-4o) 대비 파인튜닝 성능:")
    gpt4o_wins = 0
    
    for metric in key_metrics:
        ft_score = ft_results[metric]
        gpt4o_score = gpt4o_results[metric]
        diff = ft_score - gpt4o_score
        
        status = "🟢 우수" if diff > 2 else "🟡 동등" if abs(diff) <= 2 else "🔴 열세"
        print(f"  {metric}: {diff:+.1f} ({status})")
        
        if diff > 0:
            gpt4o_wins += 1
    
    # ========================================================================================
    # 8. 모델별 강점 분석
    # ========================================================================================
    
    print(f"\n⚡ 모델별 강점 분석")
    print("=" * 80)
    
    # 각 모델이 가장 우수한 메트릭 찾기
    all_metrics = list(ft_results.keys())
    model_strengths = {
        '파인튜닝_모델': [],
        'GPT-4o-mini': [],
        'GPT-4o': []
    }
    
    for metric in all_metrics:
        scores = {
            '파인튜닝_모델': ft_results[metric],
            'GPT-4o-mini': mini_results[metric],
            'GPT-4o': gpt4o_results[metric]
        }
        best_model = max(scores, key=scores.get)
        model_strengths[best_model].append((metric, scores[best_model]))
    
    for model_name, strengths in model_strengths.items():
        if strengths:
            print(f"\n🏆 {model_name} 최우수 영역:")
            for metric, score in strengths[:5]:  # 상위 5개만 표시
                print(f"  • {metric}: {score:.1f}")
    
    # ========================================================================================
    # 9. 실용성 평가
    # ========================================================================================
    
    print(f"\n💼 실용성 평가")
    print("=" * 80)
    
    # JSON 생성 안정성 평가
    json_success_rates = {
        '파인튜닝': ft_results['JSON_파싱_성공률(%)'],
        '4o-mini': mini_results['JSON_파싱_성공률(%)'],
        'GPT-4o': gpt4o_results['JSON_파싱_성공률(%)']
    }
    
    print("🔧 JSON 생성 안정성:")
    for model, rate in json_success_rates.items():
        stability = "🟢 매우안정" if rate >= 90 else "🟡 보통" if rate >= 70 else "🔴 불안정"
        print(f"  {model}: {rate:.1f}% ({stability})")
    
    # ERD 설계 품질 종합
    design_quality = {
        '파인튜닝': ft_results['설계품질_점수(%)'],
        '4o-mini': mini_results['설계품질_점수(%)'],
        'GPT-4o': gpt4o_results['설계품질_점수(%)']
    }
    
    print(f"\n🎨 ERD 설계 품질:")
    for model, quality in design_quality.items():
        grade = "🟢 우수" if quality >= 80 else "🟡 양호" if quality >= 60 else "🔴 개선필요"
        print(f"  {model}: {quality:.1f}% ({grade})")
    
    # ========================================================================================
    # 10. 최종 결론 및 권장사항
    # ========================================================================================
    
    print(f"\n🏆 최종 결론 및 권장사항")
    print("=" * 80)
    
    # 파인튜닝 효과 판정
    if mini_improvements >= 3 and avg_improvement > 5:
        ft_verdict = "🟢 파인튜닝 대성공! 베이스 모델 대비 명확한 개선"
    elif mini_improvements >= 2:
        ft_verdict = "🟡 파인튜닝 효과 있음. 일부 영역에서 개선"
    else:
        ft_verdict = "🔴 파인튜닝 효과 제한적. 추가 최적화 필요"
    
    print(f"📊 파인튜닝 효과: {ft_verdict}")
    print(f"   • {mini_improvements}/{len(key_metrics)} 주요 메트릭에서 개선")
    print(f"   • 평균 개선율: {avg_improvement:+.1f}%")
    
    # GPT-4o 대비 성능
    if gpt4o_wins >= 3:
        gpt4o_verdict = "🔥 플래그십 모델을 능가하는 놀라운 성과!"
    elif gpt4o_wins >= 2:
        gpt4o_verdict = "⚡ 플래그십 모델과 경쟁 가능한 수준"
    else:
        gpt4o_verdict = "📈 플래그십 모델 대비 개선 여지 존재"
    
    print(f"\n🎯 GPT-4o 대비: {gpt4o_verdict}")
    print(f"   • {gpt4o_wins}/{len(key_metrics)} 주요 메트릭에서 우세")
    
    # 실용적 권장사항
    print(f"\n💡 실용적 권장사항:")
    
    # 가장 우수한 모델 선택
    overall_scores = {
        '파인튜닝': np.mean([ft_results[m] for m in key_metrics]),
        '4o-mini': np.mean([mini_results[m] for m in key_metrics]),
        'GPT-4o': np.mean([gpt4o_results[m] for m in key_metrics])
    }
    
    best_model = max(overall_scores, key=overall_scores.get)
    
    if best_model == '파인튜닝':
        print("  🎯 파인튜닝 모델 사용 권장:")
        print("    - ERD 특화 작업에 최적화됨")
        print("    - 비용 효율적 (GPT-4o 대비 ~90% 절약)")
        print("    - 빠른 응답 속도")
    elif best_model == 'GPT-4o':
        print("  🔥 GPT-4o 사용 권장:")
        print("    - 최고 품질의 ERD 설계")
        print("    - 복잡한 요구사항 처리 우수")
        print("    - 높은 일관성")
    else:
        print("  🤖 GPT-4o-mini 사용 권장:")
        print("    - 균형잡힌 성능")
        print("    - 적당한 비용")
        print("    - 안정적인 결과")
    
    # 비용 대비 성능 분석
    print(f"\n💰 비용 대비 성능 분석:")
    print(f"  파인튜닝 모델: 성능 {overall_scores['파인튜닝']:.1f}, 비용 ⭐")
    print(f"  GPT-4o-mini: 성능 {overall_scores['4o-mini']:.1f}, 비용 ⭐⭐")
    print(f"  GPT-4o: 성능 {overall_scores['GPT-4o']:.1f}, 비용 ⭐⭐⭐⭐⭐")
    
    roi_scores = {
        '파인튜닝': overall_scores['파인튜닝'] * 5,  # 비용이 1/5이므로 5배 가중
        '4o-mini': overall_scores['4o-mini'] * 2,   # 비용이 1/2이므로 2배 가중
        'GPT-4o': overall_scores['GPT-4o']          # 기준
    }
    
    best_roi = max(roi_scores, key=roi_scores.get)
    print(f"  💎 최고 ROI: {best_roi} (비용효율점수: {roi_scores[best_roi]:.1f})")
    
    # ========================================================================================
    # 11. 결과 저장
    # ========================================================================================
    
    try:
        # 상세 결과 저장
        detailed_results = {
            'evaluation_summary': {
                'test_samples': test_samples,
                'evaluation_date': time.strftime("%Y-%m-%d %H:%M:%S"),
                'key_findings': {
                    'finetuning_improvements': mini_improvements,
                    'avg_improvement_rate': avg_improvement,
                    'gpt4o_competitive_metrics': gpt4o_wins,
                    'best_overall_model': best_model,
                    'best_roi_model': best_roi
                }
            },
            'detailed_metrics': evaluation_results,
            'model_strengths': model_strengths
        }
        
        with open("ERD특화_3모델_성능비교_결과.json", "w", encoding="utf-8") as f:
            json.dump(detailed_results, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 상세 결과가 'ERD특화_3모델_성능비교_결과.json' 파일로 저장되었습니다.")
        
        # 요약 리포트 저장
        summary_report = f"""
ERD 특화 3모델 성능 비교 요약 리포트
=================================

📊 테스트 개요:
- 샘플 수: {test_samples}개
- 평가 일시: {time.strftime("%Y-%m-%d %H:%M:%S")}
- 평가 방식: ERD 구조적 품질 중심

🏆 주요 결과:
- 파인튜닝 효과: {mini_improvements}/{len(key_metrics)} 메트릭 개선 (평균 {avg_improvement:+.1f}%)
- GPT-4o 대비: {gpt4o_wins}/{len(key_metrics)} 메트릭 우세
- 최고 성능 모델: {best_model}
- 최고 ROI 모델: {best_roi}

📈 핵심 메트릭 비교:
"""
        
        for metric in key_metrics:
            ft_score = ft_results[metric]
            mini_score = mini_results[metric]
            gpt4o_score = gpt4o_results[metric]
            summary_report += f"- {metric}: 파인튜닝={ft_score:.1f}, 4o-mini={mini_score:.1f}, GPT-4o={gpt4o_score:.1f}\n"
        
        summary_report += f"""
💡 권장사항:
- {ft_verdict}
- {gpt4o_verdict}
- 비용 효율성: {best_roi} 모델 권장

자세한 분석 결과는 'ERD특화_3모델_성능비교_결과.json' 파일을 참조하세요.
"""
        
        with open("ERD특화_성능비교_요약.txt", "w", encoding="utf-8") as f:
            f.write(summary_report)
        
        print(f"📄 요약 리포트가 'ERD특화_성능비교_요약.txt' 파일로 저장되었습니다.")
        
    except Exception as e:
        print(f"❌ 결과 저장 실패: {e}")

# ========================================================================================
# 메인 실행 부분
# ========================================================================================

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY 환경변수를 설정해주세요.")
        exit(1)
    
    print("🚀 ERD 특화 3모델 성능 비교를 시작합니다...")
    print(f"📁 테스트 데이터: {CSV_FILE_PATH}")
    print(f"🎯 파인튜닝 모델: {FINETUNED_MODEL}")
    print(f"🤖 GPT-4o-mini: {BASELINE_MINI_MODEL}")
    print(f"🔥 GPT-4o: {BASELINE_4O_MODEL}")
    print(f"🌡️ 온도 설정: {TEMPERATURE}")
    print(f"📊 평가 방식: ERD 구조적 품질 및 정확성 중심")
    
    run_erd_specialized_evaluation()
    
    print("\n✅ ERD 특화 3모델 성능 비교가 완료되었습니다!")
    print("📋 결과 파일:")
    print("   • ERD특화_3모델_성능비교_결과.json (상세 결과)")
    print("   • ERD특화_성능비교_요약.txt (요약 리포트)")
    print("\n🎯 이제 ERD 설계에 최적화된 평가로 정확한 모델 성능을 확인할 수 있습니다!")

🚀 ERD 특화 3모델 성능 비교를 시작합니다...
📁 테스트 데이터: hehe.csv
🎯 파인튜닝 모델: ft:gpt-4o-mini-2024-07-18:test:pja-api-finetuning-model:BmOdcDUE
🤖 GPT-4o-mini: gpt-4o-mini
🔥 GPT-4o: gpt-4o
🌡️ 온도 설정: 0.2
📊 평가 방식: ERD 구조적 품질 및 정확성 중심
🎯 ERD 특화 3모델 성능 비교: 구조적 품질과 정확성 중심 평가
✅ 데이터 로드 완료: 138개 샘플
📋 CSV 컬럼: ['user_input', 'total_requirements', 'project_info', 'ERD_data']

🔄 데이터 전처리 중...

🔄 5개 샘플에 대해 3개 모델 ERD 생성 시작...


모델 추론 진행:   0%|          | 0/5 [00:00<?, ?it/s]


📝 샘플 1 처리 중...
   🎯 파인튜닝_모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행:  20%|██        | 1/5 [00:51<03:25, 51.43s/it]


📝 샘플 2 처리 중...
   🎯 파인튜닝_모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


...


📝 샘플 4 처리 중...
   🎯 파인튜닝_모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행:  80%|████████  | 4/5 [03:33<00:53, 53.48s/it]


📝 샘플 5 처리 중...
   🎯 파인튜닝_모델 추론...
   🤖 GPT-4o-mini 추론...
   🔥 GPT-4o 추론...


모델 추론 진행: 100%|██████████| 5/5 [04:27<00:00, 53.48s/it]


📊 ERD 특화 평가 결과 분석

🔍 파인튜닝_모델 분석 중...
   JSON 파싱 성공률: 100.0%
   평균 테이블 수: 5.6
   설계 품질 점수: 93.4%
   전체 구조 일치도: 12.5%

🔍 GPT-4o-mini 분석 중...
   JSON 파싱 성공률: 100.0%
   평균 테이블 수: 5.0
   설계 품질 점수: 99.7%
   전체 구조 일치도: 13.2%

🔍 GPT-4o 분석 중...
   JSON 파싱 성공률: 100.0%
   평균 테이블 수: 5.0
   설계 품질 점수: 100.0%
   전체 구조 일치도: 16.2%

📈 ERD 특화 성능 비교 분석
메트릭                  파인튜닝         4o-mini      GPT-4o       vs mini      vs 4o       
----------------------------------------------------------------------------------------------------
JSON_파싱_성공률(%)       100.0        100.0        100.0        +0.0         +0.0        
설계품질_점수(%)           93.4         99.7         100.0        -6.3         -6.6        
전체구조_일치도(%)          12.5         13.2         16.2         -0.7         -3.7        
관계_일관성(%)            100.0        95.0         100.0        +5.0         +0.0        
완성도_점수(%)            100.0        100.0        100.0        +0.0         +0.0        

🔍 세부 ERD 품질 분석

📊 구조적 품질 분석:
  평균_테이블_수: 파인튜닝=